In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

C:\Users\harid\AppData\Local\Temp\ipykernel_22044\3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [3]:
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv(r"C:\Users\harid\Downloads\kg_train.csv",encoding='latin-1')

# Reduce the training set to speed up development.

# Modify for final system
data = data.head(1000)
print(data.shape)
data.fillna("",inplace=True)

(1000, 2)


### Let's divide the training and test set into two partitions

In [4]:
from sklearn.model_selection import train_test_split

# Split data into training and test sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

print(train_data.shape, test_data.shape)


(800, 2) (200, 2)


## Data Preprocessing

In [5]:
import string
from nltk.corpus import stopwords
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - 'C:\\Users\\harid/nltk_data'
    - 'D:\\Anaconda\\nltk_data'
    - 'D:\\Anaconda\\share\\nltk_data'
    - 'D:\\Anaconda\\lib\\nltk_data'
    - 'C:\\Users\\harid\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [6]:
import re
from bs4 import BeautifulSoup

def clean_html(html):
    # Remove inline JavaScript/CSS
    html = re.sub(r'<script.*?>.*?</script>', '', html, flags=re.DOTALL)
    html = re.sub(r'<style.*?>.*?</style>', '', html, flags=re.DOTALL)

    # Remove HTML comments
    html = re.sub(r'<!--.*?-->', '', html, flags=re.DOTALL)

    # Remove remaining HTML tags
    soup = BeautifulSoup(html, "html.parser")
    return soup.get_text()

# Example usage
html_text = """<html><head><style>body{color:red;}</style></head>
<body><!-- This is a comment --><script>alert("Hello");</script><p>Hello World</p></body></html>"""

cleaned_text = clean_html(html_text)
print(cleaned_text)



Hello World


- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [7]:
import re

def clean_text(text):
    # Remove all special characters
    text = re.sub(r'[^A-Za-z\s]', '', text)

    # Remove numbers
    text = re.sub(r'\d+', '', text)

    # Remove all single characters
    text = re.sub(r'\b\w\b', '', text)

    # Remove single characters from the start
    text = re.sub(r'^\w\s+', '', text)

    # Substitute multiple spaces with single space
    text = re.sub(r'\s+', ' ', text).strip()

    # Remove prefixed 'b'
    text = re.sub(r'\bb\s+', '', text)

    # Convert to lowercase
    return text.lower()

# Example usage
sample_text = "b This is an example! 123 text with special $$ characters and numbers."
cleaned_text = clean_text(sample_text)
print(cleaned_text)


this is an example text with special characters and numbers


## Now let's work on removing stopwords
Remove the stopwords.

In [10]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    words = text.split()
    filtered_text = ' '.join([word for word in words if word.lower() not in stop_words])
    return filtered_text

# Example usage
sample_text = "This is an example text with some common stopwords"
cleaned_text = remove_stopwords(sample_text)
print(cleaned_text)


example text common stopwords


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\harid\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [11]:
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
    words = text.split()
    lemmatized_text = ' '.join([lemmatizer.lemmatize(word) for word in words])
    return lemmatized_text

# Example usage
sample_text = "The cats are running towards the jumping dogs"
lemmatized_text = lemmatize_text(sample_text)
print(lemmatized_text)


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\harid\AppData\Roaming\nltk_data...


The cat are running towards the jumping dog


## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [13]:
import pandas as pd
from collections import Counter

# Sample data (replace with actual CSV file)
data = {'label': ['ham', 'spam', 'ham', 'spam', 'ham'],
        'message': ['Hello, how are you?', 'Win a free lottery now!', 
                    'Are we meeting tomorrow?', 'Congratulations! You won a prize.', 
                    'Let’s have lunch today.']}
df = pd.DataFrame(data)

def get_top_words(messages, n=10):
    words = ' '.join(messages).split()
    word_counts = Counter(words)
    return word_counts.most_common(n)

# Extract ham and spam messages
ham_messages = df[df['label'] == 'ham']['message'].astype(str)
spam_messages = df[df['label'] == 'spam']['message'].astype(str)

top_ham_words = get_top_words(ham_messages, 10)
top_spam_words = get_top_words(spam_messages, 10)

print("Top 10 words in Ham messages:", top_ham_words)
print("Top 10 words in Spam messages:", top_spam_words)


Top 10 words in Ham messages: [('Hello,', 1), ('how', 1), ('are', 1), ('you?', 1), ('Are', 1), ('we', 1), ('meeting', 1), ('tomorrow?', 1), ('Let’s', 1), ('have', 1)]
Top 10 words in Spam messages: [('a', 2), ('Win', 1), ('free', 1), ('lottery', 1), ('now!', 1), ('Congratulations!', 1), ('You', 1), ('won', 1), ('prize.', 1)]


## Extra features

In [14]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).
money_simbol_list = "|".join(["euro","dollar","pound","€","$"])
suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

data_train['money_mark'] = data_train['preprocessed_text'].str.contains(money_simbol_list)*1
data_train['suspicious_words'] = data_train['preprocessed_text'].str.contains(suspicious_words)*1
data_train['text_len'] = data_train['preprocessed_text'].apply(lambda x: len(x)) 

data_val['money_mark'] = data_val['preprocessed_text'].str.contains(money_simbol_list)*1
data_val['suspicious_words'] = data_val['preprocessed_text'].str.contains(suspicious_words)*1
data_val['text_len'] = data_val['preprocessed_text'].apply(lambda x: len(x)) 

data_train.head()

NameError: name 'data_train' is not defined

## How would work the Bag of Words with Count Vectorizer concept?

In [19]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

# Sample data
corpus = [
    "Free money transfer available now",
    "Win a lottery now and get a free prize",
    "Bank account security update required",
    "Let's meet for lunch tomorrow",
    "Your transaction is successful"
]

# Initialize CountVectorizer
vectorizer = CountVectorizer()

# Fit and transform the corpus
X = vectorizer.fit_transform(corpus)

# Convert to DataFrame for better visualization
df_bow = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())

# Display Bag of Words representation
print(df_bow)


   account  and  available  bank  for  free  get  is  let  lottery  ...  \
0        0    0          1     0    0     1    0   0    0        0  ...   
1        0    1          0     0    0     1    1   0    0        1  ...   
2        1    0          0     1    0     0    0   0    0        0  ...   
3        0    0          0     0    1     0    0   0    1        0  ...   
4        0    0          0     0    0     0    0   1    0        0  ...   

   prize  required  security  successful  tomorrow  transaction  transfer  \
0      0         0         0           0         0            0         1   
1      1         0         0           0         0            0         0   
2      0         1         1           0         0            0         0   
3      0         0         0           0         1            0         0   
4      0         0         0           1         0            1         0   

   update  win  your  
0       0    0     0  
1       0    1     0  
2       1    0   

## TD-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Sample data
corpus = [
    "Free money transfer available now",
    "Win a lottery now and get a free prize",
    "Bank account security update required",
    "Let's meet for lunch tomorrow",
    "Your transaction is successful"
]

# Load the vectorizer
vectorizer = TfidfVectorizer()

# Vectorize the entire dataset
X_tfidf = vectorizer.fit_transform(corpus)

# Print the shape of the vectorized dataset
print(X_tfidf.shape)


(5, 24)


## And the Train a Classifier?

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

# Sample data
corpus = [
    "Free money transfer available now",
    "Win a lottery now and get a free prize",
    "Bank account security update required",
    "Let's meet for lunch tomorrow",
    "Your transaction is successful"
]

# Labels (1 for spam, 0 for ham)
labels = [1, 1, 1, 0, 0]

# Load the vectorizer
vectorizer = TfidfVectorizer()

# Vectorize the dataset
X_tfidf = vectorizer.fit_transform(corpus)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, labels, test_size=0.2, random_state=42)

# Train the classifier
classifier = MultinomialNB()
classifier.fit(X_train, y_train)

# Predict on the test set
y_pred = classifier.predict(X_test)

# Print accuracy
print("Accuracy:", accuracy_score(y_test, y_pred))


Accuracy: 1.0


### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to find the **best feature representation**.

You can work with teams of two persons (recommended).

In [27]:
import pandas as pd

# Load dataset
df = pd.read_csv(r"C:\Users\harid\Downloads\kg_test.csv")

# Display column names
print(df.columns)


Index(['text'], dtype='object')
